## 課題：武蔵野大学 Web サイトのサイトマップ抽出

- 目的：武蔵野大学のトップページ（https://www.musashino-u.ac.jp/）にアクセスし、同一ドメイン内の全リンクを辿って各ページの URL と <title> を収集し、辞書（key=URL、value=<title>）に格納して print() で表示する。
- 実行上の制約：必ず time.sleep(seconds) を使ってサーバー負荷を軽減すること。

GitHub フローの手順（ローカルでの操作例）：

```bash
# 1. main (または upstream の最新) を取得
git checkout main
git pull origin main

# 2. 新しい作業ブランチを作る
git checkout -b feat/scrape-musashino-sitemap

# 3. ノートブックを編集・保存（この assignment.ipynb を編集）
# 4. 変更をステージしてコミット
git add assignment.ipynb
git commit -m "Add crawler notebook to extract musashino site titles"

# 5. ブランチをリモートに push
git push -u origin feat/scrape-musashino-sitemap

# 6. GitHub で Pull Request を作成してレビュー・マージ
```

注意：ノートブックは差分が大きくなりがちです。必要に応じて出力を消してコミットするか、`.gitattributes` でノートブックの扱いを調整してください。

In [3]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin, urlparse
# 武蔵野大学のトップページURL
start_url = "https://www.musashino-u.ac.jp/"
# サイトマップを格納する辞書 {URL: title}
sitemap = {}
# 訪問済みURLを記録
visited = set()
def scrape_page(url):
    """指定URLの<title>と内部リンクを抽出する関数"""
    try:
        # User-Agentを設定してブラウザからのアクセスに見せる
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        # ページにアクセス
        res = requests.get(url, headers=headers, timeout=10)
        res.encoding = res.apparent_encoding
        # ステータスコードのチェック
        if res.status_code != 200:
            print(f"スキップ: {url} (ステータスコード: {res.status_code})")
            return []
        # HTMLの解析
        soup = BeautifulSoup(res.text, "html.parser")
        # タイトルを取得
        title = soup.title.string.strip() if soup.title else "（タイトルなし）"
        sitemap[url] = title
        print(f"取得: {url} - {title}")
        # 内部リンクを収集
        links = []
        for a in soup.find_all("a", href=True):
            href = a["href"].strip()
            full_url = urljoin(url, href)
            # URLの正規化（末尾のスラッシュ処理など）
            parsed_url = urlparse(full_url)
            normalized_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"
            if parsed_url.query:
                normalized_url += f"?{parsed_url.query}"
            # フラグメント（#以降）を削除
            normalized_url = normalized_url.split('#')[0]
            # 同一ドメインかつ不要な拡張子でないものを対象
            if urlparse(normalized_url).netloc == urlparse(start_url).netloc:
                if not normalized_url.lower().endswith((".pdf", ".jpg", ".jpeg", ".png", ".gif", ".mp4", ".zip", ".doc", ".docx", ".xls", ".xlsx", ".ppt", ".pptx")):
                    if normalized_url not in visited:
                        links.append(normalized_url)
        return links
    
    except Exception as e:
        print(f"エラー: {url} ({e})")
        return []
# クローリング開始
to_visit = [start_url]
max_pages = float('inf') # 取得ページ数の上限（必要に応じて調整）
count = 0
while to_visit and count < max_pages:
    # 次に訪問するURLを取得
    url = to_visit.pop(0)
    # 既に訪問済みならスキップ
    if url in visited:
        continue
    # URLを訪問済みに追加
    visited.add(url)
    count += 1
    # ページをスクレイプして新しいリンクを取得
    new_links = scrape_page(url)
    # サーバーに負荷をかけないよう待機
    time.sleep(1)
    # 新しく見つかったリンクを訪問キューに追加
    for link in new_links:
        if link not in visited and link not in to_visit:
            to_visit.append(link)
# 結果表示
print("\n=== スクレイピング結果 ===")
print(f"取得ページ数: {len(sitemap)}")
print("サイトマップ:")
print(sitemap)

取得: https://www.musashino-u.ac.jp/ - 武蔵野大学
取得: https://www.musashino-u.ac.jp/access.html - 交通アクセス | 武蔵野大学
取得: https://www.musashino-u.ac.jp/admission/request.html - 資料請求 | 入試情報 | 武蔵野大学
取得: https://www.musashino-u.ac.jp/contact.html - お問い合わせ | 武蔵野大学
取得: https://www.musashino-u.ac.jp/prospective-students.html - 武蔵野大学で学びたい方 | 武蔵野大学
取得: https://www.musashino-u.ac.jp/students.html - 在学生の方 | 武蔵野大学
取得: https://www.musashino-u.ac.jp/alumni.html - 卒業生の方 | 武蔵野大学
取得: https://www.musashino-u.ac.jp/parents.html - 保護者の方 | 武蔵野大学
取得: https://www.musashino-u.ac.jp/business.html - 企業・研究者の方 | 武蔵野大学
取得: https://www.musashino-u.ac.jp/guide/ - 大学案内 | 武蔵野大学
取得: https://www.musashino-u.ac.jp/guide/profile/ - 大学紹介 | 大学案内 | 武蔵野大学
取得: https://www.musashino-u.ac.jp/guide/activities/ - 大学の取り組み | 大学案内 | 武蔵野大学
取得: https://www.musashino-u.ac.jp/guide/campus/ - キャンパス | 大学案内 | 武蔵野大学
取得: https://www.musashino-u.ac.jp/guide/facility/ - 附置機関・センター・附属施設 | 大学案内 | 武蔵野大学
取得: https://www.musashino-u.ac.jp/guide/information/ - 情

KeyboardInterrupt: 